In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "lasso", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9195 | AUPR: 0.6507 | Acc: 95.28%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9226 | AUPR: 0.6601 | Acc: 95.30%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9270 | AUPR: 0.6766 | Acc: 95.38%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9188 | AUPR: 0.6472 | Acc: 95.16%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9237 | AUPR: 0.6647 | Acc: 95.30%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9285 | AUPR: 0.6822 | Acc: 95.45%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9285 | AUPR: 0.6822 | Acc: 95.45%
Internal test: AUC: 0.9315 | AUPR: 0.7033 | Acc: 95.26%
External test: AUC: 0.9236 | AUPR: 0.6722 | Acc: 95.23%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9202 | AUPR: 0.6537 | Acc: 95.28%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9238 | AUPR: 0.6647 | Acc: 95.30%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9276 | AUPR: 0.6787 | Acc: 95.46%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9208 | AUPR: 0.6560 | Acc: 95.28%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9245 | AUPR: 0.6695 | Acc: 95.33%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9280 | AUPR: 0.6830 | Acc: 95.52%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9280 | AUPR: 0.6830 | Acc: 95.52%
Internal test: AUC: 0.9331 | AUPR: 0.7067 | Acc: 95.62%
External test: AUC: 0.9238 | AUPR: 0.6755 | Acc: 95.24%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9252 | AUPR: 0.6604 | Acc: 95.33%
model(C=10.0)
Valid:         AUC: 0.9251 | AUPR: 0.6597 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.9250 | AUPR: 0.6593 | Acc: 95.27%
best model(C=1.0)
Valid:         AUC: 0.9252 | AUPR: 0.6604 | Acc: 95.33%
Internal test: AUC: 0.9290 | AUPR: 0.6880 | Acc: 95.16%
External test: AUC: 0.9179 | AUPR: 0.6372 | Acc: 94.97%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9101 | AUPR: 0.6446 | Acc: 95.29%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9039 | AUPR: 0.6108 | Acc: 95.01%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9172 | AUPR: 0.6534 | Acc: 95.33%
best model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9172 | AUPR: 0.6534 | Acc: 95.33%
Internal test: AUC: 0.9043 | AUPR: 0.6715 | Acc: 95.01%
External test: AUC: 0.9001 | AUPR: 0.6268 | Acc: 94.95%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9161 | AUPR: 0.6618 | Acc: 89.39%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9169 | AUPR: 0.6617 | Acc: 89.68%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9169 | AUPR: 0.6634 | Acc: 89.51%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9163 | AUPR: 0.6630 | Acc: 89.74%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9168 | AUPR: 0.6630 | Acc: 89.85%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9170 | AUPR: 0.6642 | Acc: 89.74%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9163 | AUPR: 0.6630 | Acc: 89.74%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9168 | AUPR: 0.6630 | Acc: 89.85%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9170 | AUPR: 0.6642 | Acc: 89.74%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9170 | AUPR: 0.6642 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9231 | AUPR: 0.6302 | Acc: 95.05%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9248 | AUPR: 0.6243 | Acc: 94.92%
model(kernel=linear, C=1)
Valid:         AUC: 0.9244 | AUPR: 0.6261 | Acc: 94.91%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9248 | AUPR: 0.6243 | Acc: 94.92%
Internal test: AUC: 0.9299 | AUPR: 0.6674 | Acc: 94.89%
External test: AUC: 0.9196 | AUPR: 0.6106 | Acc: 94.70%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9238 | AUPR: 0.6585 | Acc: 95.37%
model(criterion=squared_error)
Valid:         AUC: 0.9238 | AUPR: 0.6588 | Acc: 95.37%
best model(criterion=squared_error)
Valid:         AUC: 0.9238 | AUPR: 0.6588 | Acc: 95.37%
Internal test: AUC: 0.9257 | AUPR: 0.6915 | Acc: 95.43%
External test: AUC: 0.9225 | AUPR: 0.6665 | Acc: 95.20%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6322 | Acc: 95.10%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9192 | AUPR: 0.6383 | Acc: 95.13%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9209 | AUPR: 0.6402 | Acc: 95.10%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9209 | AUPR: 0.6402 | Acc: 95.10%
Internal test: AUC: 0.9247 | AUPR: 0.6794 | Acc: 94.96%
External test: AUC: 0.9153 | AUPR: 0.6202 | Acc: 94.80%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7711 | AUPR: 0.4480 | Acc: 94.96%
model(n_neighbors=5)
Valid:         AUC: 0.7942 | AUPR: 0.4972 | Acc: 95.13%
model(n_neighbors=7)
Valid:         AUC: 0.8100 | AUPR: 0.5266 | Acc: 95.19%
best model(n_neighbors=7)
Valid:         AUC: 0.8100 | AUPR: 0.5266 | Acc: 95.19%
Internal test: AUC: 0.8165 | AUPR: 0.5483 | Acc: 94.91%
External test: AUC: 0.8131 | AUPR: 0.5272 | Acc: 94.96%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8601 | AUPR: 0.5695 | Acc: 88.04%
model(criterion=log_loss)
Valid:         AUC: 0.8624 | AUPR: 0.5751 | Acc: 89.07%
model(criterion=entropy)
Valid:         AUC: 0.8624 | AUPR: 0.5751 | Acc: 89.07%
best model(criterion=log_loss)
Valid:         AUC: 0.8624 | AUPR: 0.5751 | Acc: 89.07%
Internal test: AUC: 0.8781 | AUPR: 0.6228 | Acc: 90.75%
External test: AUC: 0.8681 | AUPR: 0.5884 | Acc: 89.99%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8886 | AUPR: 0.4629 | Acc: 85.95%
best model()
Valid:         AUC: 0.8886 | AUPR: 0.4629 | Acc: 85.95%
Internal test: AUC: 0.9021 | AUPR: 0.5157 | Acc: 87.40%
External test: AUC: 0.8921 | AUPR: 0.4570 | Acc: 86.19%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)